Imports

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import helpers.evaluate
from helpers.model_list import model_map

#Import models
models = {name: model_class() for name, model_class in model_map.items()}

Load in Data

In [8]:
df = pd.read_csv("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
X = df.drop(columns=["Diabetes_binary"])
y = df["Diabetes_binary"]
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


Explore Data

In [9]:
#TODO

Clean Data

In [10]:
#TODO

Split Data

In [11]:
#Transform target
le = LabelEncoder()
y = le.fit_transform(y)

#Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

#Scale Data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Reassign col names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)


Train Models

In [12]:


total_metrics = {}

#train each model
for name, model in models.items():
    model.train(X_train_scaled, y_train)
    metrics = model.predict(X_test_scaled, y_test)
    total_metrics[name] = metrics
    helpers.evaluate.print_stats(metrics, name)


Decision Tree  Accuracy Score:  0.6551260741945751
Decision Tree  Precision Score:  0.6560548199436097
Decision Tree  Recalle Score:  0.650713299585826
Decision Tree  F1 Score:  0.6533731428165209
Logistic Regression  Accuracy Score:  0.7476040598366164
Logistic Regression  Precision Score:  0.738359201773836
Logistic Regression  Recalle Score:  0.7662218131615278
Logistic Regression  F1 Score:  0.7520325203252033
K Nearest Neighbors  Accuracy Score:  0.6974396152349966
K Nearest Neighbors  Precision Score:  0.6841722336133342
K Nearest Neighbors  Recalle Score:  0.7323445077701866
K Nearest Neighbors  F1 Score:  0.7074392600064972
SVC  Accuracy Score:  0.7438200657778407
SVC  Precision Score:  0.7184493761310601
SVC  Recalle Score:  0.8010549045983929
SVC  F1 Score:  0.7575067786964818
AdaBoost  Accuracy Score:  0.7463309403402059
AdaBoost  Precision Score:  0.7374876157288784
AdaBoost  Recalle Score:  0.7641686431378102
AdaBoost  F1 Score:  0.7505910987482615
Bagging  Accuracy Score:

Report Stats

In [13]:
#Get best metrics
bestModel = ""
bestPerformance = 0
for name, metrics in total_metrics.items():
    if(metrics[0] > bestPerformance):
        bestModel = name
        bestPerformance = metrics[0]

print("Best Model = ", bestModel)
print("Model Accuracy = ", bestPerformance)


Best Model =  Gradient Boosting
Model Accuracy =  0.7520422958588252


Hyperparameter Tuning

In [14]:
svc = models["SVC"]
svc.tune_hyperParams(X_test_scaled, y_test)

NameError: name 'optuna' is not defined